<a href="https://colab.research.google.com/github/DamnCrab/Real-ESRGAN-Inference-Demo/blob/main/Real_ESRGAN_Inference_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-ESRGAN Inference Demo

[![arXiv](https://img.shields.io/badge/arXiv-Paper-<COLOR>.svg)](https://arxiv.org/abs/2107.10833)
[![GitHub Stars](https://img.shields.io/github/stars/xinntao/Real-ESRGAN?style=social)](https://github.com/xinntao/Real-ESRGAN)
[![download](https://img.shields.io/github/downloads/xinntao/Real-ESRGAN/total.svg)](https://github.com/xinntao/Real-ESRGAN/releases)

This is a **Practical Image Restoration Demo** of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).
We extend the powerful ESRGAN to a practical restoration application (namely, Real-ESRGAN), which is trained with pure synthetic data. <br>
The following figure shows some real-life examples.

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

We provide a pretrained model (*RealESRGAN_x4plus.pth*) with upsampling X4.<br>
**Note that RealESRGAN may still fail in some cases as the real-world degradations are really too complex.**<br>
Moreover, it **may not** perform well on **human faces, text**, *etc*, which will be optimized later.
<br>

You can also find a **Portable Windows/Linux/MacOS executable files for Intel/AMD/Nvidia GPU.** in our [GitHub repo](https://github.com/xinntao/Real-ESRGAN). <br>
This executable file is **portable** and includes all the binaries and models required. No CUDA or PyTorch environment is needed.<br>
This executable file is based on the wonderful [Tencent/ncnn](https://github.com/Tencent/ncnn) and [realsr-ncnn-vulkan](https://github.com/nihui/realsr-ncnn-vulkan) by [nihui](https://github.com/nihui).

# 1. Preparations
Before start, make sure that you choose
* Runtime Type = Python 3
* Hardware Accelerator = GPU

in the **Runtime** menu -> **Change runtime type**

Then, we clone the repository, set up the envrironment, and download the pre-trained model.

1.1 显卡检测和系统资源监控

In [ ]:
#检测使用的显卡
! /opt/bin/nvidia-smi
! nvcc --version

#注册https://wandb.ai/authorize 填api 获得实时监控网址
!pip install wandb
import wandb
wandb.init()

1.2 完整FFmpeg安装 （可跳过 安装需要30min-40min 安装后可以使用硬件加速和10bit


In [1]:
!cmake --version

cmake version 3.12.0

CMake suite maintained and supported by Kitware (kitware.com/cmake).


In [ ]:
%cd /content
!sudo apt install build-essential curl cmake
!git clone --branch 缩减音频 https://github.com/DamnCrab/ffmpeg-build-script.git
#!git clone https://github.com/DamnCrab/ffmpeg-build-script.git #凑合一下 反正能用
#!git clone https://github.com/markus-perl/ffmpeg-build-script
%cd ffmpeg-build-script 
#! ./build-ffmpeg -b --enable-gpl-and-non-free 
#%cd /content
! ./build-ffmpeg --h 
! ./build-ffmpeg -b --enable-gpl-and-non-free
#! ./build-ffmpeg -b --enable-gpl-and-non-free --full-static
! ./build-ffmpeg -c  

##原项目 https://github.com/markus-perl/ffmpeg-build-script 
##使用原项目需要修改build-ffmpeg文件
##1. 首行添加 
## AUTOINSTALL=yes   （自动跳过确认
## OSTYPE=linux-gnu
##2. 637行删除libwebp的部分    （不兼容
##3. 魔改685行 强行加入nvcc

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
curl is already the newest version (7.58.0-2ubuntu3.13).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Cloning into 'ffmpeg-build-script'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 993 (delta 77), reused 83 (delta 42), pack-reused 867
Receiving objects: 100% (993/993), 312.21 KiB | 15.61 MiB/s, done.
Resolving deltas: 100% (601/601), done.
/content/ffmpeg-build-script
ffmpeg-build-script v1.28

Usage: build-ffmpeg [OPTIONS]
Options:
  -h, --help                     Display usage information
      --version                  Display version information
  -b, --build                    Starts the build process
      --enable-gpl-and-non-

In [3]:
%cd /content
!rm -rf ffmpeg-build-script

/content


In [ ]:
!ffmpeg -version #安装成功应为ffmpeg v1.28-14-gddce366
%shell ffmpeg -hwaccels -h #显示可用硬解

1.3 Real-ESRGAN下载

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
%cd /content
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
# !pip install -r requirements.txt
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

# 2. Upload Images

Upload the images to be processed by Real-ESRGAN

2.1 网页上传或者Drive上传 （建议Drive 网页上传经常会很慢

In [ ]:
#网页上传方式
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# upload images
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)



In [ ]:
#Drive上传方式 需要授权
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import files
import shutil
upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

filename='1.mkv' #填文件名########################################################################

dst_path1 = os.path.join("/content/drive/MyDrive/", filename)

shutil.copy(dst_path1,"/content/Real-ESRGAN/upload/")




'/content/Real-ESRGAN/upload/1.mkv'

In [ ]:
# 检测输入文件FPS。
tmp_path = os.path.join("/content/Real-ESRGAN/upload/", filename)
import cv2

video = cv2.VideoCapture(tmp_path)

fps = video.get(cv2.CAP_PROP_FPS)
print("帧率: {0}".format(fps))
frame_num = video.get(cv2.CAP_PROP_FRAME_COUNT)
print("帧数: {0}".format(frame_num))

#av1编码有bug 检测不到



帧率: 60.0
帧数: 2395.0


In [ ]:
#更新ffmpeg  （PAA更新FFmpeg4 已废弃
#!add-apt-repository ppa:jonathonf/ffmpeg-4 -y
#!apt-get update
#!apt-get install ffmpeg

In [ ]:
%shell ffmpeg -i '/content/Real-ESRGAN/upload/{filename}' -codec copy -map 0 -f segment -segment_list out.csv -segment_frames 1000,2000,3000,4000,5000 out%03d.mkv

In [ ]:
!apt-get install nodejs-dev node-gyp libssl1.0-dev
!apt-get install npm
!$ npm install --global ffmpeg-progressbar-cli


In [ ]:
! npm install --global ffmpeg-progressbar-cli

In [ ]:
%shell ffprobe -v error -show_streams -print_format json '/content/Real-ESRGAN/upload/{filename}'  

In [ ]:
!ffmpeg -h encoder=libx264

In [ ]:
!ffmpeg -h encoder=libx265

In [ ]:
%shell ffmpeg -codecs

In [ ]:
%shell ffmpeg -codecs | grep nv

ffmpeg -h encoder=h264_nvenc

In [ ]:
%shell ffmpeg -hwaccels 

ffmpeg version v1.28-14-gddce366 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --enable-nonfree --enable-gpl --enable-lv2 --enable-openssl --enable-libsvtav1 --enable-libx264 --enable-libx265 --enable-libvpx --enable-libxvid --enable-libvidstab --enable-libaom --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libmp3lame --enable-libopus --enable-libvorbis --enable-libtheora --enable-libfdk-aac --enable-cuda-nvcc --enable-cuvid --enable-nvenc --enable-cuda-llvm --nvccflags='-gencode arch=compute_52,code=sm_52' --disable-debug --disable-doc --disable-shared --enable-pthreads --enable-static --enable-small --enable-cuvid --enable-version3 --extra-cflags='-I/content/ffmpeg-build-script/workspace/include -I/content/ffmpeg-build-script/workspace/include/lilv-0 -I/usr/local/cuda/include' --extra-ldexeflags=-static --extra-ldflags='-L/content/ffmpeg-build-script/workspace/lib -L/zlib/lib -L/usr/local/cuda/lib64' --e

In [ ]:
#拆帧
#filename='1.mkv'
%shell rm -rf '/content/Real-ESRGAN/frame'
%shell mkdir -p '/content/Real-ESRGAN/frame'
%shell ffmpeg -hwaccel cuvid -i '/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'


ffmpeg version v1.28-14-gddce366 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --enable-nonfree --enable-gpl --enable-lv2 --enable-openssl --enable-libsvtav1 --enable-libx264 --enable-libx265 --enable-libvpx --enable-libxvid --enable-libvidstab --enable-libaom --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libmp3lame --enable-libopus --enable-libvorbis --enable-libtheora --enable-libfdk-aac --enable-cuda-nvcc --enable-cuvid --enable-nvenc --enable-cuda-llvm --nvccflags='-gencode arch=compute_52,code=sm_52' --disable-debug --disable-doc --disable-shared --enable-pthreads --enable-static --enable-small --enable-cuvid --enable-version3 --extra-cflags='-I/content/ffmpeg-build-script/workspace/include -I/content/ffmpeg-build-script/workspace/include/lilv-0 -I/usr/local/cuda/include' --extra-ldexeflags=-static --extra-ldflags='-L/content/ffmpeg-build-script/workspace/lib -L/zlib/lib -L/usr/local/cuda/lib64' --e

CalledProcessError: ignored

# 3. Inference 


In [ ]:
#超分
# if it is out of memory, try to use the `--tile` option 爆显存调tile 现值100
!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input frame --scale 4 --half --tile 100

# 5. Download Results


In [ ]:
%shell mkdir -p '/content/Real-ESRGAN/output'

In [ ]:
%shell ffmpeg -i '/content/Real-ESRGAN/results/%08d_out.png' -c:v libx265 -preset:v [ultrafast-placebo] -profile:v main -x265-params "crf=[0-51]:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1" -pix_fmt yuv420p /content/Real-ESRGAN/output/output.mp4

In [ ]:
'/content/Real-ESRGAN/upload/{filename}' '/content/Real-ESRGAN/frame/%08d.png'

# **者佬参数**

In [ ]:
%shell mkdir -p '/content/Real-ESRGAN/output'

In [ ]:
ffmpeg

colorprim=bt709:transfer=bt709:colormatrix=bt709

-preset:v [ultrafast-placebo] -crf=[-12-51] -pix_fmt [ yuv420p / yuv420p10 ]


libx264 8bit

ffmpeg -i [.png] -c:v libx264 -preset:v [ultrafast-placebo] -profile:v high -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf=[-12-51] -x264-params "keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709" -pix_fmt yuv420p [.mp4]

libx264 10bit

ffmpeg -i [.png] -c:v libx264 -preset:v [ultrafast-placebo] -profile:v high10 -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf=[-12-51] -x264-params "keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709" -pix_fmt yuv420p10 [.mp4]

In [ ]:
%shell ffmpeg -h encoder=libx264 

In [ ]:
#%shell ffmpeg -r {fps} -y -i /content/Real-ESRGAN/frame/%08d.png -c:v libx264 -preset:v faster -profile:v high -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf 16  -pix_fmt yuv420p /content/Real-ESRGAN/output/output.mkv

In [ ]:
#264 8bit
%shell ffmpeg -r {fps} -i /content/Real-ESRGAN/frame/%08d.png -c:v libx264 -preset:v faster -profile:v high -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf 16 -x264-params keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709 -pix_fmt yuv420p /content/Real-ESRGAN/output/output.mkv

In [ ]:
#264 10bit（待修复
%shell ffmpeg -r {fps} -i /content/Real-ESRGAN/frame/%08d.png -c:v libx264 -preset:v faster -profile:v high10 -weightb 1 -weightp 2 -mbtree 1 -forced-idr 1 -coder 1 -crf 16 -x264-params keyint=200:min-keyint=1:bframes=6:b-adapt=2:no-open-gop=1:ref=8:deblock='-1:-1':rc-lookahead=50:chroma-qp-offset=-2:aq-mode=1:aq-strength=0.8:qcomp=0.75:me=umh:merange=24:subme=10:psy-rd='1:0.1':colorprim=bt709:transfer=bt709:colormatrix=bt709 -pix_fmt yuv420p10 /content/Real-ESRGAN/output/output.mkv

In [ ]:
ffmpeg

range=limited:colorprim=1:transfer=1:colormatrix=1

-preset:v [ultrafast-placebo] crf=[0-51] -pix_fmt [ yuv420p / yuv420p10 ]
ultrafast superfast veryfast faster fast medium (default) slow slower veryslow placebo
master-display="G(15000,30000)B(7500,3000)R(32000,16500)WP(15635,16450)L(10000000,50)":max-cll="1000,100"


libx265 8bit

ffmpeg -i [.png] -c:v libx265 -preset:v [ultrafast-placebo] -profile:v main -x265-params "crf=[0-51]:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1" -pix_fmt yuv420p [.mp4]

libx265 10bit

ffmpeg -i [.png] -c:v libx265 -preset:v [ultrafast-placebo] -profile:v main10 -x265-params "crf=[0-51]:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1" -pix_fmt yuv420p10 [.mp4]

libx265 10bit HDR10+

ffmpeg -i [.png] -c:v libx265 -preset:v [ultrafast-placebo] -profile:v main10 -x265-params "crf=[0-51]:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=9:transfer=16:colormatrix=9:master-display="G(15000,30000)B(7500,3000)R(32000,16500)WP(15635,16450)L(10000000,50)":max-cll="1000,100":hdr10-opt=1:repeat-headers=1" -pix_fmt yuv420p10 [.mp4]

In [ ]:
%shell ffmpeg -h encoder=libx265 

In [ ]:
%shell ffmpeg -y -r {fps} -i /content/Real-ESRGAN/frame/%08d.png -c:v libx265 -preset:v faster -x265-params crf=16:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1 -pix_fmt yuv420p /content/Real-ESRGAN/output/output.mkv

In [ ]:
%shell ffmpeg -y -r {fps} -i /content/Real-ESRGAN/frame/%08d.png -c:v libx265 -preset:v faster -profile:v main10 -x265-params crf=16:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=1:transfer=1:colormatrix=1:repeat-headers=1 -pix_fmt yuv420p10 /content/Real-ESRGAN/output/output.mkv

In [ ]:
%shell ffmpeg -y -r {fps} -i /content/Real-ESRGAN/frame/%08d.png -c:v libx265 -preset:v faster -profile:v main10 -x265-params crf=16:ref=4:rd=3:no-rect=1:no-amp=1:b-intra=1:rdoq-level=2:limit-tu=4:me=3:subme=5:weightb=1:no-strong-intra-smoothing=1:psy-rd=2.0:psy-rdoq=1.0:no-open-gop=1:keyint=200:min-keyint=1:rc-lookahead=20:bframes=6:aq-mode=1:aq-strength=0.8:qg-size=8:cbqpoffs=-2:crqpoffs=-2:qcomp=0.65:deblock=-1:no-sao=1:range=limited:colorprim=9:transfer=16:colormatrix=9:master-display="G(15000,30000)B(7500,3000)R(32000,16500)WP(15635,16450)L(10000000,50)":max-cll="1000,100":hdr10-opt=1:repeat-headers=1 -pix_fmt yuv420p10 /content/Real-ESRGAN/output/output.mkv

In [ ]:
ffmpeg

-color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv

-preset:v [12-18 (fastest-slowest)] -cq [1-51] -pix_fmt [ yuv420p / p010le ]

 -cq                <float>      E..V....... Set target quality level (0 to 51, 0 means automatic) for constant quality mode in VBR rate control (from 0 to 51) (default 0)
-preset            <int>        E..V....... Set the encoding preset (from 0 to 18) (default p4)
     default         0            E..V....... 
     slow           1            E..V....... hq 2 passes
     medium          2            E..V....... hq 1 pass
     fast           3            E..V....... hp 1 pass
     hp            4            E..V....... 
     hq            5            E..V....... 
     bd            6            E..V....... 
     ll            7            E..V....... low latency
     llhq           8            E..V....... low latency hq
     llhp           9            E..V....... low latency hp
     lossless         10           E..V....... 
     losslesshp        11           E..V....... 
     p1              12           E..V....... fastest (lowest quality)
     p2              13           E..V....... faster (lower quality)
     p3              14           E..V....... fast (low quality)
     p4              15           E..V....... medium (default)
     p5              16           E..V....... slow (good quality)
     p6              17           E..V....... slower (better quality)
     p7              18           E..V....... slowest (best quality)

h264_nvenc 8bit

ffmpeg -i [.png] -flags "cgop" -g 200 -bf 4 -i_qfactor 0.71 -b_qfactor 1.3 -keyint_min 1 -c:v h264_nvenc -preset:v [12-18 (fastest-slowest)] -tune:v 1 -profile:v 2 -cq [1-51] -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -temporal-aq 1 -coder 1 -b_ref_mode 2 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt yuv420p 264.mp4

hevc_nvenc 8bit

ffmpeg -i [.png] -flags "cgop" -g 200 -i_qfactor 0.71 -keyint_min 1 -c:v hevc_nvenc -preset:v [12-18 (fastest-slowest)] -tune:v 1 -profile:v 0 -cq [1-51] -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt yuv420p 265-8.mp4

-bf 0（5代）
-bf 0 -weighted_pred 1 （6代）
-bf 5 -b_qfactor 1.3 -temporal-aq 1 -b_ref_mode 2 （7代）

hevc_nvenc 10bit

ffmpeg -i [.png] -flags "cgop" -g 200 -i_qfactor 0.71 -keyint_min 1 -c:v hevc_nvenc -preset:v [12-18 (fastest-slowest)] -tune:v 1 -profile:v 1 -cq [1-51] -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt p010le 265-10.mp4

-bf 0 -weighted_pred 1 （6代）
-bf 5 -b_qfactor 1.3 -temporal-aq 1 -b_ref_mode 2 （7代）

In [ ]:
%shell ffmpeg -h encoder=h264_nvenc

In [ ]:
%shell ffmpeg -y -r {fps} -i /content/Real-ESRGAN/frame -flags "cgop" -g 200 -bf 4 -i_qfactor 0.71 -b_qfactor 1.3 -keyint_min 1 -c:v h264_nvenc -preset:v 18 -tune:v 1 -profile:v 2 -cq 16 -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -temporal-aq 1 -coder 1 -b_ref_mode 2 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt yuv420p /content/Real-ESRGAN/output/output.mkv

In [ ]:
%shell ffmpeg -y -r {fps} -i /content/Real-ESRGAN/frame -flags "cgop" -g 200 -i_qfactor 0.71 -keyint_min 1 -c:v hevc_nvenc -preset:v 18 -tune:v 1 -profile:v 0 -cq 16 -rc-lookahead 50 -forced-idr 1 -spatial-aq 1 -color_primaries bt709 -color_trc bt709 -colorspace bt709 -color_range tv -pix_fmt yuv420p /content/Real-ESRGAN/output/output.mkv

In [ ]:
# Download the results
zip_filename = 'Real-ESRGAN_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} results/*")
files.download(zip_filename)

In [ ]:
%shell ffmpeg -hwaccel_device 0 -y -r 30 -i /content/Real-ESRGAN/frame/%08d.png  -c:v -hwaccel cuvid  /content/Real-ESRGAN/resultsoutput.mkv

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe91e115350>> (for pre_run_cell):


Exception: ignored

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fe91e115350>> (for post_run_cell):


Exception: ignored